In [ ]:
# Source-to-parcel analysis

# Import necessary libraries
from matplotlib.animation import FuncAnimation
# import seaborn as sns  # required for heatmap visualization
import networkx as nx
from scipy.stats import pearsonr
from mne.viz import circular_layout
import pandas as pd
from mne_connectivity.viz import plot_connectivity_circle
import matplotlib.pyplot as plt
import os
import glob
import numpy as npc
import cupy as np #using gpu acceleration
import cupyx.scipy.fft
import mne
from mne.datasets import fetch_fsaverage
from nilearn import datasets
from nilearn.image import get_data
from scipy.signal import hilbert #scipy core modified in env, running custom lib
import scipy
import matplotlib
import os.path as op

matplotlib.use('Agg')  # Setting the backend BEFORE importing pyplot



scipy.fft.set_backend(cupyx.scipy.fft)

fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")


# Import necessary Python modules
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')


# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


def compute_cross_correlation(data_window):
    """Compute cross-correlation for given data window."""
    # Reshape the data to be 2D
    
    
    data_2D = data_window.reshape(data_window.shape[0], -1)
    correlation_matrix = np.corrcoef(data_2D, rowvar=True)
    return correlation_matrix

        # Compute dPLI at the level of regions


def compute_dPLI(data):
    print('Computing dPLI')
    n_regions = data.shape[1]  # Compute for regions
    dPLI_matrix = np.zeros((n_regions, n_regions))
    print(data)
    analytic_signal = hilbert(data)
    phase_data = np.angle(analytic_signal)
    for i in range(n_regions):
        for j in range(n_regions):
            if i != j:
                phase_diff = phase_data[:, i] - phase_data[:, j]
                dPLI_matrix[i, j] = np.abs(
                    np.mean(np.exp(complex(0, 1) * phase_diff)))
    return dPLI_matrix

# dPLI_matrix = compute_dPLI(label_time_courses) --> computing static, fc for the entire dataset


def disparity_filter(G, alpha=0.01):
    disparities = {}
    for i, j, data in G.edges(data=True):
        weight_sum_square = sum(
            [d['weight']**2 for _, _, d in G.edges(i, data=True)])
        disparities[(i, j)] = data['weight']**2 / weight_sum_square

    G_filtered = G.copy()
    for (i, j), disparity in disparities.items():
        if disparity < alpha:
            G_filtered.remove_edge(i, j)
    return G_filtered


def graph_to_matrix(graph, size):
    matrix = np.zeros((size, size))
    for i, j, data in graph.edges(data=True):
        matrix[i, j] = data['weight']
        matrix[j, i] = data['weight']  # Ensure symmetry
    return matrix

def plot_matrix(matrix, title, labels, cmap='viridis'):
    plt.figure(figsize=(10, 10))
    #plt.imsave(matrix, cmap='autumn')
    sns.heatmap(matrix, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.savefig(output_path+title+'.png')

def update(window_number):
    ax.clear()
    current_matrix = graph_to_matrix(nx.convert_matrix.from_numpy_array(
        windowed_cross_correlation_matrices[window_number]), windowed_cross_correlation_matrices[window_number].shape[0])
    sns.heatmap(current_matrix, cmap='viridis',
                xticklabels=ordered_regions, yticklabels=ordered_regions, ax=ax)
    title.set_text(
        f'Thresholded Cross-Correlation Matrix for Window {window_number}')
    return ax

def update(window_number):
    ax.clear()
    current_matrix = threshold_matrix(windowed_dpli_matrices[window_number])
    plot_connectivity_circle(current_matrix, ordered_regions, n_lines=300, node_angles=node_angles,
                            title=f'Thresholded Regional Connectivity using dPLI for Window {window_number}', ax=ax)
    return ax,


def threshold_matrix(matrix):
    G_temp = nx.convert_matrix.from_numpy_array(matrix)
    G_temp_thresholded = disparity_filter(G_temp)

    matrix_thresholded = np.zeros_like(matrix)
    for i, j, data in G_temp_thresholded.edges(data=True):
        matrix_thresholded[i, j] = data['weight']
        matrix_thresholded[j, i] = data['weight']
    return matrix_thresholded

def threshold_graph_by_density(G, density=0.1, directed=False):
    if density < 0 or density > 1:
        raise ValueError("Density value must be between 0 and 1.")
    num_edges_desired = int(G.number_of_edges() * density)
    sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'], 
reverse=True)
    if directed:
        G_thresholded = nx.DiGraph()
    else:
        G_thresholded = nx.Graph()
    G_thresholded.add_edges_from(sorted_edges[:num_edges_desired])
    return G_thresholded


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}


schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100)
fs_dir = '../data/in/fsaverage'
fname = os.path.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
src = mne.read_source_spaces(fname, patch_stats=False, verbose=None)

for subject in subject_list:
    for mode in modes:
        print(subject, mode)
        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        stc_path  = output_path +'stc/'

        inverse_solution_files_lh = []
        inverse_solution_files_rh = []
        
        
        for path, subdirs, files in os.walk(stc_path):
            for file in files:
                filepath = path + file
                if '-rh.stc' in file:
                    inverse_solution_files_rh.append(filepath)
                elif '-lh.stc' in file:
                    inverse_solution_files_lh.append(filepath)

        # Error here !!!

        # Calculate the total number of inverse solution files for both hemispheres
        total_files_lh = len(inverse_solution_files_lh)
        total_files_rh = len(inverse_solution_files_rh)

        # Calculate the batch size for both hemispheres
        # Change 10 to your desired batch size
        batch_size_lh = total_files_lh // (total_files_lh // 10)

        # Change 10 to your desired batch size
        batch_size_rh = total_files_rh // (total_files_rh // 10)

        # Ensure batch size is a multiple of 10 (or your desired batch size) for both hemispheres
        while total_files_lh % batch_size_lh != 0:
            batch_size_lh -= 1

        while total_files_rh % batch_size_rh != 0:
            batch_size_rh -= 1

        # Initialize lists to store source estimates for both hemispheres
        stcs_lh = []
        stcs_rh = []

        # Load data in batches for both hemispheres
        for i in range(0, total_files_lh, batch_size_lh):
            batch_files_lh = inverse_solution_files_lh[i:i + batch_size_lh]
            batch_files_rh = inverse_solution_files_rh[i:i + batch_size_rh]

            for file_path_lh, file_path_rh in zip(batch_files_lh, batch_files_rh):
                try:
                    
                    stc_epoch_lh = mne.read_source_estimate(file_path_lh)
                    stc_epoch_rh = mne.read_source_estimate(file_path_rh)
                    stcs_lh.append(stc_epoch_lh)
                    stcs_rh.append(stc_epoch_rh)
                except Exception as e:
                    print(f"Error loading files {file_path_lh} or {file_path_rh}: {e}")

        # Load labels from the atlas
        labels = mne.read_labels_from_annot('fsaverage', parc='Schaefer2018_100Parcels_7Networks_order',
                                            subjects_dir=r'../data/in/')

        # Extract label time courses for both hemispheres
        label_time_courses = []  # Initialize a list to store label time courses
        
        
        for idx, (stc_lh, stc_rh) in enumerate(zip(stcs_lh, stcs_rh)):
            try:
                label_tc_lh = stc_lh.extract_label_time_course(
                    labels, src=src, mode='mean_flip')
                label_tc_rh = stc_rh.extract_label_time_course(
                    labels, src=src, mode='mean_flip')
                label_time_courses.extend([label_tc_lh, label_tc_rh])
                
            except Exception as e:
                print(f"Error extracting label time courses for iteration {idx}: {e}")
        else:  # This block will execute if the for loop completes without encountering a break statement
            print("All time courses have been successfully extracted!")

        # Convert label_time_courses to a NumPy array
        label_time_courses_np = np.array(label_time_courses)

        # If you prefer to save as a .csv file
        # Convert to DataFrame and save as .csv
        # label_time_courses_df = pd.DataFrame(label_time_courses_np)
        # label_time_courses_df.to_csv(os.path.join(output_dir, f"{subj}_label_time_courses.csv"), index=False)

        # Save the label time courses as a .npy file
        # Replace with your desired output directory
        label_time_courses_file = output_path + f"{subject}_label_time_courses.npy"
        
        np.save(label_time_courses_file, label_time_courses_np)
        continue

0 files missing from root.txt in /home/prakhar7/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/prakhar7/mne_data/MNE-fsaverage-data/fsaverage
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
101 EC
Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extr

Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102

Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102

Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102

Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102 labels (mode: mean_flip)
Extracting time courses for 102

101 EC
Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsav

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

...

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

Reading labels from parcellation...
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/lh.Schaefer2018_100Parcels_7Networks_order.annot
   read 51 labels from /home/prakhar7/mfprl/sample/neuroconn/batch/../data/in/fsaverage/label/rh.Schaefer2018_100Parcels_7Networks_order.annot
[<Label | fsaverage, '7Networks_LH_Cont_Cing_1-lh', lh : 2109 vertices>, <Label | fsaverage, '7Networks_LH_Cont_PFCl_1-lh', lh : 3157 vertices>, <Label | fsaverage, '7Networks_LH_Cont_Par_1-lh', lh : 3556 vertices>, <Label | fsaverage, '7Networks_LH_Cont_pCun_1-lh', lh : 1267 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_1-lh', lh : 1975 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_2-lh', lh : 3328 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_3-lh', lh : 3389 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_4-lh', lh : 2603 vertices>, <Label | fsaverage, '7Networks_LH_Default_PFC_5-lh', lh : 4594 vertices>, <Label | fsaverage, 

KeyboardInterrupt: 